<a href="https://colab.research.google.com/github/xinrongl/BaiduImageSpider/blob/master/swimming_pool_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and split data to train and val

In [89]:
!wget -O data.zip "https://storage.googleapis.com/kaggle-data-sets/142930/335089/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1582848450&Signature=EMBBXAEWXHiHPwkObCp74kpYQJ8vLEZ9raURL5aJl9KfzvcTGZ69EKvkRbMHBWkTqVAVaNh6ClwpWgPVDRlAxt14YnPojrlbV23SrzwuMDQEo%2F4Q6knHB7rJdYL7o4gJErsGD87KeJjP6zclswXYCvlpLnYcNj4zUZHinyD0bgYr0GaQsmr%2FL0u6nsNvdfHXoMWDg8LrS2gEfPwojyckrzULgPsfKHaEkt%2BQ5jTB3AHmY9Hg7bLqtOBnqKMOIS4W%2FXXgFwS2HwgvicOcxns3%2BK9AOHMESnc0KTDwCJzwMM0UJNOpfir1%2B3%2FMeuXwmUiILk7WaiVzqyxoWwDSQXaHKg%3D%3D&response-content-disposition=attachment%3B+filename%3Dswimming-pool-and-car-detection.zip"
!unzip data.zip

--2020-02-25 01:52:43--  https://storage.googleapis.com/kaggle-data-sets/142930/335089/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1582848450&Signature=EMBBXAEWXHiHPwkObCp74kpYQJ8vLEZ9raURL5aJl9KfzvcTGZ69EKvkRbMHBWkTqVAVaNh6ClwpWgPVDRlAxt14YnPojrlbV23SrzwuMDQEo%2F4Q6knHB7rJdYL7o4gJErsGD87KeJjP6zclswXYCvlpLnYcNj4zUZHinyD0bgYr0GaQsmr%2FL0u6nsNvdfHXoMWDg8LrS2gEfPwojyckrzULgPsfKHaEkt%2BQ5jTB3AHmY9Hg7bLqtOBnqKMOIS4W%2FXXgFwS2HwgvicOcxns3%2BK9AOHMESnc0KTDwCJzwMM0UJNOpfir1%2B3%2FMeuXwmUiILk7WaiVzqyxoWwDSQXaHKg%3D%3D&response-content-disposition=attachment%3B+filename%3Dswimming-pool-and-car-detection.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.208, 2607:f8b0:4007:803::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116621313 (111M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[================

In [0]:
from pathlib import Path, PurePath
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import shutil

In [0]:
root = Path(".")
train_path = root.joinpath("training_data")
images_path = sorted(list(train_path.rglob("*/images/*.jpg")))
labels_path = sorted(list(train_path.rglob("*/labels/*.xml")))
assert len(images_path) == len(labels_path)

In [0]:
def parse_annotation(annotation):

    tree = ET.parse(annotation)
    root = tree.getroot()

    boxes, labels = [], []
    for object in root.iter("object"):
        label = object.find('name').text.lower().strip()
        bbox = object.find('bndbox')
        xmin = int(float(bbox.find('xmin').text))
        ymin = int(float(bbox.find('ymin').text))
        xmax = int(float(bbox.find('xmax').text))
        ymax = int(float(bbox.find('ymax').text))

        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(int(label))

    return {'boxes': boxes, 'labels': labels}


def train_val_split(inputs_dir, outputs_dir, split_ratio=0.9):
    if not isinstance(inputs_dir, PurePath):
        inputs_dir = Path(inputs_dir)
    if not isinstance(outputs_dir, PurePath):
        outputs_dir = Path(outputs_dir)

    train_dir, val_dir = outputs_dir.joinpath('train'), outputs_dir.joinpath('val')
    
    paths  = []
    for d in [train_dir, val_dir]:
        for c in ['0', '1']:
            path = d.joinpath(c)
            path.mkdir(parents=True, exist_ok=True)
            paths.append(path)
    train_pos, train_neg, val_pos, val_neg = paths

    images = list(sorted(inputs_dir.rglob('*/images/*.jpg')))
    labels = list(sorted(inputs_dir.rglob('*/labels/*.xml')))
    assert len(images)==len(labels)
    
    train_pos_count = 0
    train_neg_count = 0
    val_pos_count = 0
    val_neg_count = 0
    threshold = int(split_ratio*len(images))
    
    for i, (image, label) in enumerate(zip(images, labels)):
        objects = parse_annotation(label)
        fn = image.parts[-1]
        if i <= threshold:
            if 2 in objects['labels']: # label for swimming pool is 2
                shutil.copy(image, train_pos.joinpath(fn))
                train_pos_count += 1
            else:
                shutil.copy(image, train_neg.joinpath(fn))
                train_neg_count += 1
        else:
            if 2 in objects['labels']:
                shutil.copy(image, val_pos.joinpath(fn))
                val_pos_count += 1
            else:
                shutil.copy(image, val_neg.joinpath(fn))
                val_neg_count += 1
        print('Processed %s images' % (i+1))

    print("\nDone")
    print('Process %s images' % len(images))
    print('Positive train: %s' % train_pos_count)
    print('Negative train: %s' % train_neg_count)
    print('Positive val  : %s' % val_pos_count)
    print('Negative val  : %s' % val_neg_count)
    return None

In [93]:
train_val_split(inputs_dir='training_data', outputs_dir="data")

Processed 1 images
Processed 2 images
Processed 3 images
Processed 4 images
Processed 5 images
Processed 6 images
Processed 7 images
Processed 8 images
Processed 9 images
Processed 10 images
Processed 11 images
Processed 12 images
Processed 13 images
Processed 14 images
Processed 15 images
Processed 16 images
Processed 17 images
Processed 18 images
Processed 19 images
Processed 20 images
Processed 21 images
Processed 22 images
Processed 23 images
Processed 24 images
Processed 25 images
Processed 26 images
Processed 27 images
Processed 28 images
Processed 29 images
Processed 30 images
Processed 31 images
Processed 32 images
Processed 33 images
Processed 34 images
Processed 35 images
Processed 36 images
Processed 37 images
Processed 38 images
Processed 39 images
Processed 40 images
Processed 41 images
Processed 42 images
Processed 43 images
Processed 44 images
Processed 45 images
Processed 46 images
Processed 47 images
Processed 48 images
Processed 49 images
Processed 50 images
Processed

## Simple Image Classifier

In [100]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np  # linear algebra
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
import sys
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn as nn
from tqdm import tqdm, trange
import time
from PIL import ImageFile

seed = 42
# BATCH_SIZE = 2 ** 5
BATCH_SIZE = 8
NUM_WORKERS = 4
LEARNING_RATE = 5e-5
LR_STEP = 5
LR_FACTOR = 0.1
NUM_EPOCHS = 15
LOG_FREQ = 20
RESIZE = 224
WD = 0

torch.cuda.empty_cache()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.backends.cudnn.benchmark = True
model_path = f'./models/train_simple_{RESIZE}/'
os.system(f'mkdir -p {model_path}')

0

In [0]:
# Data loading code
class ImageDataset(Dataset):
    def __init__(self, data_dir, mode):
        assert mode in ['train', 'val', 'test']
        self.mode = mode
        self.fn = list(Path(data_dir).rglob('*.jpg'))
        transforms_list = [
            transforms.Resize(RESIZE),
            transforms.CenterCrop(RESIZE)
        ]
        if self.mode == 'train':
            transforms_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomChoice([
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
                    transforms.RandomAffine(degrees=(0, 360), translate=(0.1, 0.1), scale=(0.8, 1.2),
                                            resample=Image.BILINEAR),
                    transforms.RandomGrayscale()
                ])
            ])
        transforms_list.extend(
            [transforms.ToTensor()]
        )
        self.transforms = transforms.Compose(transforms_list)

    def __getitem__(self, index):
        label = self.fn[index].parts[-2]
        label = np.array(int(label))
        image = Image.open(self.fn[index])
        assert image.mode == 'RGB'
        image = self.transforms(image)
        return image, label

    def __len__(self):
        return len(self.fn)


class AverageMeter:
    ''' Computes and stores the average and current value '''

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def train(train_loader, model, criterion, optimizer, epoch, logging=True):
    batch_time = AverageMeter()
    losses = AverageMeter()
    avg_f1 = AverageMeter()
    avg_auc = AverageMeter()
    avg_acc = AverageMeter()

    model.train()
    num_steps = len(train_loader)

    end = time.time()
    lr_str = ''

    for i, (input_, targets) in enumerate(train_loader):
        if i >= num_steps:
            break

        output = model(input_.to(device))
        loss = criterion(output, targets.to(device))

        _, predicts = torch.max(output.data, 1)
        predicts = predicts.cpu().numpy()
        targets = targets.cpu().numpy()
        avg_f1.update(f1_score(targets, predicts, average='micro'))
        avg_acc.update(accuracy_score(targets, predicts))
        # if not (targets == 1).all() or (targets == 0).all():
        #     avg_auc.update(roc_auc_score(targets, output.detach().cpu().numpy()[:, 1], average='micro'))

        losses.update(loss.data.item(), input_.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if logging and i % LOG_FREQ == 0:
            print(f'{epoch} [{i}/{num_steps}]\t'
                  f'time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'loss {losses.val:.4f} ({losses.avg:.4f})\t'
                  f'F1 {avg_f1.val:.4f} ({avg_f1.avg:.4f})\t'
                  f'accuracy {avg_acc.val:.4f} ({avg_acc.avg:.4f})\t'
                  # f'auc {avg_auc.val:.4f} ({avg_auc.avg:.4f})\t'
                  + lr_str)
            sys.stdout.flush()

    print(f' * average F1 on train {avg_f1.avg:.4f}')
    print(f' * average Accuracy on train {avg_acc.avg:.4f}')
    # print(f' * average AUC on train {avg_auc.avg:.4f}')
    if epoch > 1:
        torch.save(
            {
                'epoch': epoch,
                'arch': 'resnext101_32x8d',
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }, f"{model_path}/resnext_{epoch}.pth")
    return avg_acc.avg


def inference(data_loader, model):
    ''' Returns predictions and targets, if any. '''
    model.eval()

    all_predicts, all_confs, all_targets = [], [], []
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            input_, target = data

            output = model(input_.to(device))
            all_confs.append(output)
            _, predicts = torch.max(output.data, 1)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets


def validate(val_loader, model):
    predicts, confs, targets = inference(val_loader, model)
    predicts = predicts.cpu().numpy()
    confs = confs.cpu().numpy()
    targets = targets.cpu().numpy()

    f1 = f1_score(targets, predicts, average='micro')
    acc = accuracy_score(targets, predicts)
    # if not (targets == 1).all() or (targets == 0).all():
    #     auc = roc_auc_score(targets, confs[:, 1], average='micro')

    print(f"val f1: {f1:.4f}")
    print(f"val accuracy: {acc:.4f}")
    # print(f"val auc {auc:.4f}")

    return acc


def train_loop(epochs, train_loader, val_loader, model, criterion, optimizer, lr_scheduler):
    train_res = []
    val_res = []
    for epoch in trange(1, epochs + 1):
        print(f"learning rate: {lr_scheduler.get_lr()}")
        start_time = time.time()
        train_score = train(train_loader, model, criterion,
                            optimizer, epoch, logging=True)

        train_res.append(train_score)
        lr_scheduler.step()

        val_acc = validate(val_loader, model)
        val_res.append(val_acc)
        print(f"epoch {epoch} validation accuracy: {val_acc:.4f}\n")

    return np.asarray(train_res), np.asarray(val_res)

In [0]:
train_dataset = ImageDataset("./data/train", mode='train')
val_dataset = ImageDataset("./data/val", mode='val')

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          drop_last=True,
                          num_workers=NUM_WORKERS)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=NUM_WORKERS)

In [101]:
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
model.fc = nn.Linear(model.fc.in_features, 2)

model = nn.DataParallel(model)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE,
                             weight_decay=WD)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=LR_STEP,
                                               gamma=LR_FACTOR)

train_res, val_res = train_loop(NUM_EPOCHS, train_loader, val_loader, model,
                                criterion, optimizer, lr_scheduler)

print(f"best val score/epoch: {np.max(val_res):.4f};{np.argmax(val_res) + 1}")

Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master

  0%|          | 0/15 [00:00<?, ?it/s]

learning rate: [5e-05]
1 [0/421]	time 0.840 (0.840)	loss 0.7024 (0.7024)	F1 0.3750 (0.3750)	accuracy 0.3750 (0.3750)	
1 [20/421]	time 0.479 (0.497)	loss 0.7450 (0.6768)	F1 0.6250 (0.6071)	accuracy 0.6250 (0.6071)	
1 [40/421]	time 0.479 (0.488)	loss 0.5021 (0.5900)	F1 0.7500 (0.6646)	accuracy 0.7500 (0.6646)	
1 [60/421]	time 0.480 (0.486)	loss 0.7543 (0.5546)	F1 0.7500 (0.6885)	accuracy 0.7500 (0.6885)	
1 [80/421]	time 0.482 (0.485)	loss 0.5246 (0.5263)	F1 0.7500 (0.7160)	accuracy 0.7500 (0.7160)	
1 [100/421]	time 0.482 (0.484)	loss 0.0931 (0.4908)	F1 1.0000 (0.7438)	accuracy 1.0000 (0.7438)	
1 [120/421]	time 0.479 (0.483)	loss 0.6663 (0.4662)	F1 0.7500 (0.7634)	accuracy 0.7500 (0.7634)	
1 [140/421]	time 0.480 (0.483)	loss 0.1386 (0.4535)	F1 1.0000 (0.7757)	accuracy 1.0000 (0.7757)	
1 [160/421]	time 0.480 (0.482)	loss 0.3050 (0.4432)	F1 0.8750 (0.7811)	accuracy 0.8750 (0.7811)	
1 [180/421]	time 0.479 (0.482)	loss 0.4639 (0.4415)	F1 0.7500 (0.7873)	accuracy 0.7500 (0.7873)	
1 [200/421]	t


  7%|▋         | 1/15 [03:29<48:50, 209.33s/it]

val f1: 0.9118
val accuracy: 0.9118
epoch 1 validation accuracy: 0.9118

learning rate: [5e-05]
2 [0/421]	time 0.685 (0.685)	loss 0.2517 (0.2517)	F1 1.0000 (1.0000)	accuracy 1.0000 (1.0000)	
2 [20/421]	time 0.483 (0.493)	loss 0.4760 (0.2759)	F1 0.7500 (0.9048)	accuracy 0.7500 (0.9048)	
2 [40/421]	time 0.486 (0.489)	loss 0.1197 (0.2800)	F1 1.0000 (0.8933)	accuracy 1.0000 (0.8933)	
2 [60/421]	time 0.485 (0.487)	loss 0.0412 (0.2885)	F1 1.0000 (0.8893)	accuracy 1.0000 (0.8893)	
2 [80/421]	time 0.483 (0.487)	loss 0.0613 (0.2864)	F1 1.0000 (0.8904)	accuracy 1.0000 (0.8904)	
2 [100/421]	time 0.483 (0.486)	loss 0.0834 (0.2911)	F1 1.0000 (0.8899)	accuracy 1.0000 (0.8899)	
2 [120/421]	time 0.484 (0.486)	loss 0.4238 (0.3085)	F1 0.8750 (0.8771)	accuracy 0.8750 (0.8771)	
2 [140/421]	time 0.481 (0.485)	loss 0.1458 (0.3137)	F1 1.0000 (0.8768)	accuracy 1.0000 (0.8768)	
2 [160/421]	time 0.486 (0.485)	loss 0.4916 (0.3339)	F1 0.7500 (0.8634)	accuracy 0.7500 (0.8634)	
2 [180/421]	time 0.484 (0.485)	loss 0


 13%|█▎        | 2/15 [07:02<45:37, 210.59s/it]

...

val f1: 0.9465
val accuracy: 0.9465
epoch 13 validation accuracy: 0.9465

learning rate: [5.000000000000001e-07]
14 [0/421]	time 0.804 (0.804)	loss 0.2058 (0.2058)	F1 0.8750 (0.8750)	accuracy 0.8750 (0.8750)	
14 [20/421]	time 0.489 (0.502)	loss 0.2380 (0.1169)	F1 0.8750 (0.9583)	accuracy 0.8750 (0.9583)	
14 [40/421]	time 0.489 (0.495)	loss 0.0396 (0.1185)	F1 1.0000 (0.9543)	accuracy 1.0000 (0.9543)	
14 [60/421]	time 0.490 (0.493)	loss 0.0706 (0.1182)	F1 1.0000 (0.9529)	accuracy 1.0000 (0.9529)	
14 [80/421]	time 0.490 (0.492)	loss 0.0526 (0.1223)	F1 1.0000 (0.9506)	accuracy 1.0000 (0.9506)	
14 [100/421]	time 0.488 (0.491)	loss 0.0980 (0.1292)	F1 1.0000 (0.9468)	accuracy 1.0000 (0.9468)	
14 [120/421]	time 0.486 (0.490)	loss 0.2050 (0.1351)	F1 0.8750 (0.9473)	accuracy 0.8750 (0.9473)	
14 [140/421]	time 0.487 (0.490)	loss 0.0318 (0.1297)	F1 1.0000 (0.9504)	accuracy 1.0000 (0.9504)	
14 [160/421]	time 0.488 (0.489)	loss 0.0525 (0.1298)	F1 1.0000 (0.9495)	accuracy 1.0000 (0.9495)	
14 [180/421


 93%|█████████▎| 14/15 [49:56<03:34, 214.63s/it]

val f1: 0.9519
val accuracy: 0.9519
epoch 14 validation accuracy: 0.9519

learning rate: [5.000000000000001e-07]
15 [0/421]	time 0.805 (0.805)	loss 0.2594 (0.2594)	F1 0.8750 (0.8750)	accuracy 0.8750 (0.8750)	
15 [20/421]	time 0.488 (0.501)	loss 0.2476 (0.1330)	F1 0.8750 (0.9524)	accuracy 0.8750 (0.9524)	
15 [40/421]	time 0.486 (0.494)	loss 0.0409 (0.1084)	F1 1.0000 (0.9634)	accuracy 1.0000 (0.9634)	
15 [60/421]	time 0.489 (0.492)	loss 0.4094 (0.1486)	F1 0.8750 (0.9406)	accuracy 0.8750 (0.9406)	
15 [80/421]	time 0.489 (0.491)	loss 0.2127 (0.1581)	F1 0.8750 (0.9367)	accuracy 0.8750 (0.9367)	
15 [100/421]	time 0.487 (0.491)	loss 0.0923 (0.1550)	F1 1.0000 (0.9394)	accuracy 1.0000 (0.9394)	
15 [120/421]	time 0.486 (0.490)	loss 0.2829 (0.1491)	F1 0.8750 (0.9421)	accuracy 0.8750 (0.9421)	
15 [140/421]	time 0.487 (0.490)	loss 0.3464 (0.1432)	F1 0.8750 (0.9441)	accuracy 0.8750 (0.9441)	
15 [160/421]	time 0.487 (0.489)	loss 0.1450 (0.1424)	F1 1.0000 (0.9457)	accuracy 1.0000 (0.9457)	
15 [180/421


100%|██████████| 15/15 [53:31<00:00, 214.71s/it]


val f1: 0.9465
val accuracy: 0.9465
epoch 15 validation accuracy: 0.9465

best val score/epoch: 0.9519;10
